In [8]:
import os
import torch
import copy
import tensorflow as tf
import numpy as np
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
# from Downloads import LSTMSudokuClassifier as LSTMC
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from random import randint

In [9]:
class LSTMClassifier(nn.Module):

    def __init__(self, input_dim, hidden_dim, n_label, batch_size, n_layers, use_gpu):
        super(LSTMClassifier, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.use_gpu = use_gpu
        self.n_layers = n_layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, n_layers, bidirectional=True)
        self.hidden2label = nn.Sequential(torch.nn.Linear(2*hidden_dim, n_label),
                                          torch.nn.Softmax(dim = 2))
        self.hidden = self.init_hidden()

    def init_hidden(self):
        if self.use_gpu:
            h0 = Variable(torch.zeros(2*n_layers, self.batch_size, self.hidden_dim).cuda())
            c0 = Variable(torch.zeros(2*n_layers, self.batch_size, self.hidden_dim).cuda())
        else:
            h0 = Variable(torch.zeros(2*n_layers, self.batch_size, self.hidden_dim))
            c0 = Variable(torch.zeros(2*n_layers, self.batch_size, self.hidden_dim))
        return (h0, c0)
    
    def ManipulateSquare(self, lstm_out_square):
        counter = 0
        inner_stack = None
        outer_stack = None
        for i in range(3):
            for j in range(3):
                x = lstm_out_square[counter,:,:]
                counter += 1
                x = x.view(3,3,2*hidden_dim)#change to 2*hidden_dim
                if inner_stack is None:
                    inner_stack = x
                else:
                    inner_stack = torch.cat((inner_stack,x),1)        
            if outer_stack is None:
#                 print(inner_stack.shape)
                outer_stack = inner_stack
                inner_stack = None
#                 print('B = ',outer_stack)
            else:
#                 print(inner_stack.shape)
                outer_stack = torch.cat((outer_stack,inner_stack),0)
#                 print('B = ',outer_stack)
                inner_stack = None
        return outer_stack

    def forward(self, row_tensor, col_tensor, square_tensor):
#         row = row_tensor.view(self.n_features, self.batch_size, self.input_dim) #May not be necessary
#         col = col_tensor.view(self.n_features, self.batch_size, self.input_dim) #May not be necessary
#         square = square_tensor.view(self.n_features, self.batch_size, self.input_dim) #May not be necessary
        lstm_out_row, hidden_row = self.lstm(row_tensor, self.hidden)
        lstm_out_col, hidden_col = self.lstm(col_tensor, self.hidden)
        lstm_out_square, hidden_square = self.lstm(square_tensor, self.hidden) 
        # Do manipulation Here
        #Row & Col Manipulation
#         print(lstm_out_row.shape, lstm_out_row)
#         print(lstm_out_col.shape, lstm_out_col)
        row_col = lstm_out_row.data.cpu().numpy() + np.transpose(lstm_out_col.data.cpu().numpy(), (1, 0, 2))
#         print("row+col", row_col.shape, row_col)
        #Square Manipulation
        final_square = self.ManipulateSquare(lstm_out_square)
        lstm_out = row_col + final_square.data.cpu().numpy()
#         lstm_out = lstm_out_row.data.cpu().numpy()
#         print("row+col+sqaure", lstm_out.shape, lstm_out)
        lstm_out_tensor = Variable(torch.FloatTensor(lstm_out))
        if self.use_gpu:
            lstm_out_tensor = Variable(torch.FloatTensor(lstm_out).cuda())
#         print("final tensor", lstm_out_tensor.shape, lstm_out_tensor)
        
        #Sum Square Row Col
        y  = self.hidden2label(lstm_out_tensor)
#         print('Output Shape ', y.shape)
        return y

In [10]:
class InputGrid():
    
    def __init__(self, Grid):
        self.Grid = Grid
        self.gridLength = len(Grid[0])
#         self.labels = labels
        
    def getlims(self, i):
        if 0 <= i <= 2:
            rowlims = [0,3]
        elif 3 <= i <= 5:
            rowlims = [3,6]
        elif 6 <= i <= 8:
                rowlims = [6,9]
        return rowlims

    def get_square(self, i):
        grid_square = None
        if i < 3:
            if i%3 == 0:
                grid_square = self.Grid[0:3,0:3]
            elif i%3 == 1:
                grid_square = self.Grid[0:3,3:6]
            elif i%3 == 2:
                grid_square = self.Grid[0:3,6:9]
        elif i < 6:
            if i%3 == 0:
                grid_square = self.Grid[3:6,0:3]
            elif i%3 == 1:
                grid_square = self.Grid[3:6,3:6]
            elif i%3 == 2:
                grid_square = self.Grid[3:6,6:9]
        elif i < 9:
            if i%3 == 0:
                grid_square = self.Grid[6:9,0:3]
            elif i%3 == 1:
                grid_square = self.Grid[6:9,3:6]
            elif i%3 == 2:
                grid_square = self.Grid[6:9,6:9]
        return grid_square.flatten()
                
    
    # takes 1D returns 2D
    def one_hot(self, vec):
        one_hot_matrix = []
        for val in vec:
            hot_vec = [0 for _ in range(9)]
            if val > 0:
                hot_vec[int(val)-1] = 1
            else:
                hot_vec = [1.0/9 for _ in range(9)]
            one_hot_matrix.append(hot_vec)
        real_one_hot = np.array(one_hot_matrix)
        return real_one_hot
    
    #takes 2D returns 3D
    def getInput(self):
#         print(self.Grid)
        Rows = np.zeros((9,9,9))
        Columns = np.zeros((9,9,9))
        Squares = np.zeros((9,9,9))
        Labels = np.zeros((9,9,9))
        for i in range(self.gridLength):
            hot_row = self.one_hot(self.Grid[i,:])
            hot_column = self.one_hot(self.Grid[:,i])
            hot_square = self.one_hot(self.get_square(i))
#             hot_label = self.one_hot(self.labels[i,:])
            
            Rows[i] = hot_row
            Columns[i] = hot_column
            Squares[i] = hot_square
#             Labels[i] = hot_label
        
        row_tensor = Variable(torch.FloatTensor(Rows))
        col_tensor = Variable(torch.FloatTensor(Columns))
        square_tensor = Variable(torch.FloatTensor(Squares))
#         label_tensor = Variable(torch.LongTensor(Labels))
#         print(row_tensor)
#         print(col_tensor)
#         print(square_tensor)
#         print(label_tensor)
                 
        return row_tensor, col_tensor, square_tensor

In [11]:
# use_plot = True
# use_save = True
# if use_save:
#     import pickle
#     from datetime import datetime

DATA_DIR = 'data'
TRAIN_FILE = 'sudoku_dev.csv'
TEST_FILE = 'sudoku_test.txt'
TRAIN_LABEL = 'train_label.txt'
TEST_LABEL = 'test_label.txt'

In [12]:
class SudokuDataset(Dataset):
    def __init__(self, fpath):
        
        print(fpath)
        lines = open(fpath, 'r').read().splitlines()[1:]
        nsamples = len(lines)

        X = np.zeros((nsamples, 9*9), np.float32)  
        Y = np.zeros((nsamples, 9*9), np.int32) 

        for i, line in enumerate(lines):
            quiz, solution = line.split(",")
            for j, (q, s) in enumerate(zip(quiz, solution)):
                X[i, j], Y[i, j] = q, s

        X = np.reshape(X, (-1, 9, 9))
        Y = np.reshape(Y, (-1, 9, 9))
        self.X = X
        self.Y = Y

    def __getitem__(self, index):
        quiz = self.X[index]
        sol = self.Y[index]
        return quiz, sol
    
    def __len__(self):
        return len(self.X)

In [13]:
if __name__=='__main__':
    
    ## parameter setting
    epochs = 100
    batch_size = 9
    use_gpu = torch.cuda.is_available()
    learning_rate = 1e-5

    input_dim = 9
    hidden_dim = 100
    n_label = 9
    n_layers = 5
    
    train_path = os.path.join(DATA_DIR, TRAIN_FILE)
    test_path = os.path.join(DATA_DIR, TEST_FILE)


     ### ********************create model**************************
    model = LSTMClassifier(input_dim, hidden_dim, n_label, batch_size, n_layers, use_gpu)
    if use_gpu:
        model = model.cuda()
    
    training_set = SudokuDataset(train_path)
    train_loader = DataLoader(training_set,
                          batch_size=1,
                          shuffle=True,
                          num_workers=0
                          )

    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    loss_function = nn.CrossEntropyLoss()
    train_loss_ = []
    test_loss_ = []
    train_acc_ = []
    test_acc_ = []
    
### training procedure
    for epoch in range(epochs):
        # optimizer = adjust_learning_rate(optimizer, epoch)
        total_acc = 0.0
        total_loss = 0.0
        total = 0.0
        
#         model.hidden = model.init_hidden()
        for iter, traindata in enumerate(train_loader):
            train_inputs_raw, train_labels_raw = traindata

            train_data_numpy = Variable(train_inputs_raw).data.numpy()
            train_labels_numpy = Variable(train_labels_raw).data.numpy()
            train_labels = Variable(torch.LongTensor(train_labels_numpy[0]));
    
            input_grid = InputGrid(train_data_numpy[0])
            row_tensor, col_tensor, square_tensor = input_grid.getInput()

            if use_gpu:
                row_tensor, col_tensor, square_tensor, train_labels = \
                        row_tensor.cuda(), col_tensor.cuda(), square_tensor.cuda(), train_labels.cuda()

            model.hidden = model.init_hidden()
            model.batch_size = len(train_labels)     
            output = model(row_tensor, col_tensor, square_tensor)
            
            loss = loss_function(output.view(81,9), train_labels.view(81,))
#             print("loss btw:", output.view(81,9), train_labels.view(81,))
            loss.backward()
            optimizer.step()
            model.zero_grad()
            

            # calc training acc
            _, predicted = torch.max(output.data, 1)
            predicted = predicted.add_(1)
#             print('TrainPred =  ',predicted, 'While TrainLabel = ' ,train_labels.data)
            total_acc += (predicted.view(81,) == train_labels.data.view(81,)).sum()
#             print((predicted.view(81,) == train_labels.data.view(81,)).sum())
            total += 81
#             print(train_labels.data)
            total_loss += loss.data[0]

#        print("loss:", loss.data[0], "acc", total_acc)
            train_loss_.append(total_loss / total)
            train_acc_.append(total_acc / total)

        print('[Epoch: %3d/%3d] Training Loss: %.3f, Training Acc: %.3f' 
                  % (epoch, epochs, train_loss_[epoch], train_acc_[epoch]))
        

data\sudoku_dev.csv
[Epoch:   0/100] Training Loss: 0.027, Training Acc: 0.123
[Epoch:   1/100] Training Loss: 0.027, Training Acc: 0.123
[Epoch:   2/100] Training Loss: 0.027, Training Acc: 0.123
[Epoch:   3/100] Training Loss: 0.027, Training Acc: 0.120
[Epoch:   4/100] Training Loss: 0.027, Training Acc: 0.119
[Epoch:   5/100] Training Loss: 0.027, Training Acc: 0.121
[Epoch:   6/100] Training Loss: 0.027, Training Acc: 0.123
[Epoch:   7/100] Training Loss: 0.027, Training Acc: 0.125
[Epoch:   8/100] Training Loss: 0.027, Training Acc: 0.126
[Epoch:   9/100] Training Loss: 0.027, Training Acc: 0.126


KeyboardInterrupt: 

In [ ]:
## parameter setting
epochs = 50
batch_size = 1
use_gpu = torch.cuda.is_available()
learning_rate = 0.01

def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate * (0.1 ** (epoch // 10))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    return optimizer